In [2]:
from IPython.core.display import HTML
HTML('<style>.container {width:95% !important; ) </style>')

import pandas   as pd
import sqlite3  as lite
import numpy    as np
import datetime as dt
from NBA_df_scripts import (getNBAteamdata, getNBAplayerdata, getsportsbookdata, merge_team_line_data, team_feature_generation)
con = lite.connect(r'/Users/rossmcdonald/projects/NBA Daily Fantasy Data/nba_df.db')
cur = con.cursor()
%matplotlib inline

/Users/rossmcdonald/Library/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Team data build assembles the final Team data frame to be incorporated into Model building. I should Write that data  

In [19]:
def team_data_build(db_con):
    
    # I need to update all of these functions to delete and update the most recent season of data.
    
    # read team from DB
    teambox = pd.read_sql('SELECT * FROM team_box_scores;',db_con) 
    teambox['GAME_DATE'] = teambox['GAME_DATE'].apply(pd.to_datetime)
    
    # read line data from DB
    nbalines = pd.read_sql('SELECT * FROM line_results;',db_con)   
    nbalines['date'] = nbalines['date'].apply(pd.to_datetime) 
 
    teamdf = team_feature_generation(merge_team_line_data(line_data = nbalines,team_data = teambox))
    
    return teamdf

In [20]:
teamdf = team_data_build(db_con = con)

NBA_df_scripts.py:217: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=1,window=82,center=False).mean(how=down)
  lambda x: pd.rolling_mean(x,window=82,min_periods=1,how="down").shift(1))
NBA_df_scripts.py:219: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=1,window=82,center=False).mean(how=down)
  lambda x: pd.rolling_mean(x,window=82,min_periods=1,how="down").shift(1))
NBA_df_scripts.py:221: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=1,window=82,center=False).mean(how=down)
  lambda x: pd.rolling_mean(x,window=82,min_periods=1,how="down").shift(1))
NBA_df_scripts.py:223: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=1

In [21]:
# Prep data set to merge in Travel distance data
teamdf = teamdf.set_index(['season','game_num','Team_ID','Location'], drop = False)
teamdf['LAST_OPP_ID'] = teamdf.sort_index(ascending=True).groupby(level = [0,2])['Team_ID_OPP'].shift(1)
teamdf['GAME_LOC_ID'] = np.where(teamdf['Location'] == 'Home', teamdf['Team_ID'], teamdf['Team_ID_OPP'])
teamdf['LAST_GAME_LOC_ID'] = np.where(teamdf['LAST_GAME_Location'] == 'Home', teamdf['Team_ID'], teamdf['LAST_OPP_ID']) 

In [64]:
# Read travel distance data from DB
Travel_Distance = pd.read_sql('SELECT * FROM Travel_Distance;',con)

# Merge in travel distance
finaldf =  teamdf.merge(Travel_Distance[['travel_distance_km','team_id_home','team_id_away']],
            how = 'left', left_on= ['GAME_LOC_ID','LAST_GAME_LOC_ID'], right_on = ['team_id_home','team_id_away'])

del finaldf['team_id_home']; del finaldf['team_id_away']; #del finaldf['GAME_LOC_ID']; del finaldf['LAST_GAME_LOC_ID']

# Merge in Opponent Travel distance
merge_fields = finaldf[["Team_ID","Game_ID","travel_distance_km"]]

teamfinal = finaldf.merge(merge_fields, how = 'left', on = ['Game_ID'], suffixes=("", "_OPP_P")).query(
    "Team_ID != Team_ID_OPP").set_index(['season','game_num','Team_ID','Location'],drop = False)

del teamfinal['Team_ID_OPP_P']

teamfinal = teamfinal.rename(columns = {'travel_distance_km_OPP_P':'travel_distance_km_OPP'})

# Useful variables to build the model off of.  

# Final step is to make a final DF in our DB, and write code to run an efficient update on that table 

In [65]:
cur.execute('DROP TABLE team_feature_data')

In [66]:
teamfinal.to_sql('team_feature_data',con=con,schema='dbo',if_exists = 'append',index=False)

In [61]:
finaldf = pd.read_sql('SELECT * FROM team_feature_data',con=con)

In [70]:
print sorted(finaldf.columns) ,

['AST', 'AST_OPP', 'AVG_AST', 'AVG_AST_OPP', 'AVG_BLK', 'AVG_BLK_OPP', 'AVG_DREB', 'AVG_DREB_OPP', 'AVG_FGA', 'AVG_FGA_OPP', 'AVG_FGM', 'AVG_FGM_OPP', 'AVG_FTA', 'AVG_FTA_OPP', 'AVG_FTM', 'AVG_FTM_OPP', 'AVG_OREB', 'AVG_OREB_OPP', 'AVG_PACE', 'AVG_PACE_OPP', 'AVG_POSS', 'AVG_POSS_OPP', 'AVG_PTS', 'AVG_PTS_OPP', 'AVG_REB', 'AVG_REB_OPP', 'AVG_STL', 'AVG_STL_OPP', 'AVG_TOV', 'AVG_TOV_OPP', 'BLK', 'BLK_OPP', 'DAYS_OFF', 'DAYS_OFF_OPP', 'DREB', 'DREB_OPP', 'FG3A', 'FG3M', 'FG3_PCT', 'FGA', 'FGA_OPP', 'FGM', 'FGM_OPP', 'FG_PCT', 'FTA', 'FTA_OPP', 'FTM', 'FTM_OPP', 'FT_PCT', 'GAME_DATE', 'GAME_LOC_ID', 'GameOutome', 'Game_ID', 'L', 'LAST_GAME', 'LAST_GAME_LOC_ID', 'LAST_GAME_Location', 'LAST_OPP_ID', 'Line_Pt_Prediction', 'Location', 'MATCHUP', 'MIN', 'OREB', 'OREB_OPP', 'OT', 'OverUnderLine', 'OverUnderOutcome', 'PACE', 'PF', 'POSS', 'PTS', 'PTS_OPP', 'Points_Opponent', 'Points_Team', 'REB', 'REB_OPP', 'STL', 'STL_OPP', 'Spread', 'SpreadOutcome', 'TEAM_NAME', 'TOV', 'TOV_OPP', 'TRAVEL_FLAG'

In [34]:
outcome_vars = ['REB','AST','STL','BLK','TOV','PTS']

input_vars = [  'season'            ,'team_OPP'     ,'TEAM_NAME'           , # team variables to use
'AVG_PTS_OPP'  ,'AVG_REB_OPP'       ,'AVG_BLK_OPP'  ,'AVG_AST_OPP'         , # AVG Stats Opponent
'AVG_STL_OPP'  ,'AVG_TOV_OPP'       ,'AVG_FGM_OPP'  ,'AVG_FGA_OPP'         ,
'AVG_FTM_OPP'  ,'AVG_FTA_OPP'       ,'AVG_OREB_OPP' ,'AVG_DREB_OPP'        ,
'AVG_PACE_OPP' ,'AVG_POSS_OPP'      ,'AVG_PTS'      ,'AVG_REB'             , #AVG Stats Team
'AVG_BLK'      ,'AVG_AST'           ,'AVG_STL'      ,'AVG_TOV'             ,
'AVG_FGM'      ,'AVG_FGA'           ,'AVG_FTA'      ,'AVG_FTM'             ,
'AVG_OREB'     ,'AVG_DREB'          ,'AVG_PACE'     ,'AVG_POSS'            , 
'DAYS_OFF'     ,'TRAVEL_FLAG'       ,'Location'     ,'travel_distance_km'  ,     # General Game Information 
'SpreadOutcome','Line_Pt_Prediction','Spread'       ,'travel_distance_km_OPP' ] # Vegas information 

In [13]:
finaldf[['travel_distance_km','GAME_LOC_ID','LAST_GAME_LOC_ID','LAST_OPP_ID','team_OPP','Team_ID_OPP','Team_ID',
         'date_as_int','TEAM_NAME','Location','POSS']][finaldf['Team_ID']==1610612758].head(10)

,travel_distance_km,GAME_LOC_ID,LAST_GAME_LOC_ID,LAST_OPP_ID,team_OPP,Team_ID_OPP,Team_ID,date_as_int,TEAM_NAME,Location,POSS
0,581.117361,1.610613e+09,1.610613e+09,1.610613e+09,Los Angeles Lakers,1.610613e+09,1.610613e+09,20020417,Kings,Away,98.234476
1,0.000000,1.610613e+09,1.610613e+09,1.610613e+09,Golden State Warriors,1.610613e+09,1.610613e+09,20020416,Kings,Home,101.526186
2,581.117361,1.610613e+09,1.610613e+09,1.610613e+09,Dallas Mavericks,1.610613e+09,1.610613e+09,20020414,Kings,Home,90.386826
3,552.529654,1.610613e+09,1.610613e+09,1.610613e+09,Los Angeles Clippers,1.610613e+09,1.610613e+09,20020412,Kings,Away,90.310897
4,109.872535,1.610613e+09,1.610613e+09,1.610613e+09,Golden State Warriors,1.610613e+09,1.610613e+09,20020410,Kings,Away,103.054595
5,0.000000,1.610613e+09,1.610613e+09,1.610613e+09,Minnesota Timberwolves,1.610613e+09,1.610613e+09,20020409,Kings,Home,95.463297
6,858.252280,1.610613e+09,1.610613e+09,1.610613e+09,New York Knicks,1.610613e+09,1.610613e+09,20020407,Kings,Home,97.030342
7,2401.527260,1.610613e+09,1.610613e+09,1.610613e+09,Utah Jazz,1.610613e+09,1.610613e+09,20020405,Kings,Away,96.971660
8,1002.375834,1.610613e+09,1.610613e+09,1.610613e+09,Detroit Pistons,1.610613e+09,1.610613e+09,20020403,Kings,Away,94.702121
9,542.829684,1.610613e+09,1.610613e+09,1.610613e+09,Memphis Grizzlies,1.610613e+09,1.610613e+09,20020402,Kings,Away,98.595180


In [15]:
finaldf[finaldf['POSS'] < 50] 

,level_0,index,index_x,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,...,POSS,PACE,AVG_PACE,AVG_POSS,AVG_PACE_OPP,AVG_POSS_OPP,LAST_OPP_ID,GAME_LOC_ID,LAST_GAME_LOC_ID,travel_distance_km
38954,38954,38954,0,1.610613e+09,0021600089,2016-11-06 00:00:00,SAC @ TOR,None,None,None,...,7.416667,inf,92.282630,93.698223,93.676980,93.676980,1.610613e+09,1.610613e+09,1.610613e+09,694.134950
39025,39025,39025,66,1.610613e+09,0021600089,2016-11-06 00:00:00,TOR vs. SAC,None,None,None,...,7.416667,inf,93.676980,93.676980,92.282630,93.698223,1.610613e+09,1.610613e+09,1.610613e+09,0.000000
39555,39555,39555,47,1.610613e+09,0021600356,2016-12-11 00:00:00,DET vs. PHI,None,None,None,...,38.655000,154.620000,93.458957,93.458957,96.294362,98.026230,1.610613e+09,1.610613e+09,1.610613e+09,874.501506
39609,39609,39609,101,1.610613e+09,0021600356,2016-12-11 00:00:00,PHI @ DET,None,None,None,...,38.655000,154.620000,96.294362,98.026230,93.458957,93.458957,1.610613e+09,1.610613e+09,1.610613e+09,811.470289
39610,39610,39610,101,1.610613e+09,0021600356,2016-12-11 00:00:00,PHI @ DET,None,None,None,...,38.655000,154.620000,96.294362,98.026230,93.458957,93.458957,1.610613e+09,1.610613e+09,1.610613e+09,1510.906445
39716,39716,39716,67,1.610613e+09,0021600561,2017-01-08 00:00:00,LAC vs. MIA,None,None,None,...,37.059783,148.239130,95.566721,96.103532,93.792366,94.593214,1.610613e+09,1.610613e+09,1.610613e+09,581.117361
39996,39996,39996,337,1.610613e+09,0021600561,2017-01-08 00:00:00,MIA @ LAC,None,None,None,...,37.059783,148.239130,93.792366,94.593214,95.566721,96.103532,1.610613e+09,1.610613e+09,1.610613e+09,0.000000
